In [2]:
# from youtube_comment_scraper_python import *
# import pandas as pd
# link = 'https://www.youtube.com/watch?v=uYPbbksJxIg&ab_channel=UniversalPictures'
# youtube.open(link)
# response = youtube.video_comments()
# all_data = []
# for i in range(0, 10): # It will scroll 10 times
#     response = youtube.video_comments()
#     data = response['body']
#     all_data.extend(data)
# df = pd.DataFrame(data)
# df.to_csv('./data/comments.csv')

# Datatransformation

* **Libraries required**

In [3]:
#Libraries 
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
%matplotlib inline
import os

# Import functions for data preprocessing & data preparation
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import resample
from sklearn.feature_extraction.text import CountVectorizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer

from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer, LancasterStemmer
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
from nltk.corpus import wordnet
import string
from string import punctuation
import nltk
import re

* **Read data**

In [4]:
data = pd.read_csv('./data/comments.csv')
data.columns
data1=data.drop(['Unnamed: 0','UserLink','Likes','Time','user'],axis=1)
data1

,Comment
0,"""You're the man who gave them the power to des..."
1,I loved the part when Thomas Shelby retired fr...
2,"""Until somebody builds a bigger bomb.""\nI love..."
3,that one second shot of einstein is just absol...
4,"I was supposed to watch this with my uncle, wh..."
...,...
175,Periodic Movie + Cillian Murphy + Nolan Execut...
176,"""Because no one cared about the bomb that didn..."
177,"""Now I am become death, the destroyer of world..."
178,the cinematography matches interstellar. I hop...


* **Data labelling**

In [5]:
nltk.download('vader_lexicon')
sentiments = SentimentIntensityAnalyzer()
data1["Positive"] = [sentiments.polarity_scores(i)["pos"] for i in data1["Comment"]]
data1["Negative"] = [sentiments.polarity_scores(i)["neg"] for i in data1["Comment"]]
data1["Neutral"] = [sentiments.polarity_scores(i)["neu"] for i in data1["Comment"]]
data1['Compound'] = [sentiments.polarity_scores(i)["compound"] for i in data1["Comment"]]
score = data1["Compound"].values
sentiment = []
for i in score:
    if i >= 0.05 :
        sentiment.append('Positive')
    elif i <= -0.05 :
        sentiment.append('Negative')
    else:
        sentiment.append('Neutral')
data1["Sentiment"] = sentiment
data1.head()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\spand\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


,Comment,Positive,Negative,Neutral,Compound,Sentiment
0,"""You're the man who gave them the power to des...",0.000,0.243,0.757,-0.6660,Negative
1,I loved the part when Thomas Shelby retired fr...,0.218,0.000,0.782,0.8442,Positive
2,"""Until somebody builds a bigger bomb.""\nI love...",0.075,0.121,0.804,-0.7017,Negative
3,that one second shot of einstein is just absol...,0.204,0.000,0.796,0.6249,Positive
4,"I was supposed to watch this with my uncle, wh...",0.141,0.073,0.786,0.7424,Positive


* **Final data**

In [6]:
data2=data1.drop(['Positive','Negative','Neutral','Compound'],axis=1)
data2.head()

,Comment,Sentiment
0,"""You're the man who gave them the power to des...",Negative
1,I loved the part when Thomas Shelby retired fr...,Positive
2,"""Until somebody builds a bigger bomb.""\nI love...",Negative
3,that one second shot of einstein is just absol...,Positive
4,"I was supposed to watch this with my uncle, wh...",Positive


* **Data transformation**

In [7]:
nltk.download('stopwords')
stop_words = stopwords.words('english')
porter_stemmer = PorterStemmer()
lancaster_stemmer = LancasterStemmer() 
snowball_stemer = SnowballStemmer(language="english")
lzr = WordNetLemmatizer()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\spand\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [8]:
def text_processing(text):   
    # convert text into lowercase
    text = text.lower()

    # remove new line characters in text
    text = re.sub(r'\n',' ', text)
    
    # remove punctuations from text
    text = re.sub('[%s]' % re.escape(punctuation), "", text)
    
    # remove references and hashtags from text
    text = re.sub("^a-zA-Z0-9$,.", "", text)
    
    # remove multiple spaces from text
    text = re.sub(r'\s+', ' ', text, flags=re.I)
    
    # remove special characters from text
    text = re.sub(r'\W', ' ', text)

    text = ' '.join([word for word in word_tokenize(text) if word not in stop_words])
    
    # stemming using porter stemmer from nltk package - msh a7sn 7aga - momken: lancaster, snowball
    text=' '.join([porter_stemmer.stem(word) for word in word_tokenize(text)])
    text=' '.join([lancaster_stemmer.stem(word) for word in word_tokenize(text)])
    text=' '.join([snowball_stemer.stem(word) for word in word_tokenize(text)])
    
    # lemmatizer using WordNetLemmatizer from nltk package
    text=' '.join([lzr.lemmatize(word) for word in word_tokenize(text)])

    return text

In [9]:
nltk.download('omw-1.4')
data_copy = data2.copy()
data_copy.Comment = data_copy.Comment.apply(lambda text: text_processing(text))

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\spand\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [10]:
le = LabelEncoder()
data_copy['Sentiment'] = le.fit_transform(data_copy['Sentiment'])

In [11]:
processed_data = {
    'Sentence':data_copy.Comment,
    'Sentiment':data_copy['Sentiment']
}

processed_data = pd.DataFrame(processed_data)
processed_data.head()

,Sentence,Sentiment
0,yo man gav pow destroy world prep giv chil,0
1,lov part thom shelb retir lif peak blind mov u...,2
2,somebod build big bomb lov lin seem giv nonch ...,0
3,on second shot einstein absolv goosebumpinduc ...,2
4,suppo watch unc taught everi oppenheim team ma...,2


In [12]:
processed_data['Sentiment'].value_counts()

Sentiment
2    114
1     43
0     23
Name: count, dtype: int64

Here,   0 - Negative; 
        1 - Neutral;
        2 - Positive

* **Balancing data**

In [13]:
df_neutral = processed_data[(processed_data['Sentiment']==1)] 
df_negative = processed_data[(processed_data['Sentiment']==0)]
df_positive = processed_data[(processed_data['Sentiment']==2)]

# upsample minority classes
df_negative_upsampled = resample(df_negative, 
                                 replace=True,    
                                 n_samples= 205, 
                                 random_state=42)  

df_neutral_upsampled = resample(df_neutral, 
                                 replace=True,    
                                 n_samples= 205, 
                                 random_state=42)  


# Concatenate the upsampled dataframes with the neutral dataframe
final_data = pd.concat([df_negative_upsampled,df_neutral_upsampled,df_positive])

In [14]:
final_data['Sentiment'].value_counts()

Sentiment
0    205
1    205
2    114
Name: count, dtype: int64

In [15]:
corpus = []
for sentence in final_data['Sentence']:
    corpus.append(sentence)
corpus[0:5]

['known oppenheim lif man year ago end mov hop leav u emot stat said would op door pow mess right credit run exampl would today 11000 nuclear weapon pow gre xv z act would hit peopl hard child earl 70 old enough rememb 1983 terrif ev led u closest ev cam novemb 21st 1983 day mad tv mov brok record turn mov terrif left ronald reag depress chang stant nuclear war led greatest reduc nuclear weapon sadl memor dont some world memor second mess mov imagin head screen point year 1945 right trinit test ai learn mistak afraid answ',
 'yo man gav pow destroy world prep absolv incredibleno word describ',
 'murph nol toge cre mast piec deserv osc sound radioact detect giv goosebump ye inv gav mankind peac nev seen also gav destruct mankind nev seen right rocket man',
 'giv cast crew big round appla sacr larg nuclear bomb explo film mak mov felt real auth',
 'iv seen ever nol film the sint batm begin ever on imax sint dark knight say iv nev excit nol film the set im count day']

In [16]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=1500)
X = cv.fit_transform(corpus).toarray()
y = final_data.iloc[:, -1].values

* **Machine learning model**

In [17]:
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
classifier = GaussianNB()
classifier.fit(X_train, y_train)

GaussianNB()

* **Evaluation**

In [18]:
from sklearn.metrics import confusion_matrix, accuracy_score
y_pred = classifier.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
cm

array([[59,  0,  0],
       [ 0, 63,  1],
       [ 8,  4, 23]], dtype=int64)

In [19]:
nb_score = accuracy_score(y_test, y_pred)
print('accuracy',nb_score)

accuracy 0.9177215189873418
